In [1]:
# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import scipy.stats as ss
import csv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pickle
from collections import defaultdict
import operator
from scipy.sparse import csr_matrix
import itertools
import os.path
import math
import pybedtools

In [2]:
# Import custom libraries
import process_hic_contacts_inter as phc

In [3]:
# Specify directories and relevant information
genome_dir = '../../data/genome_data/'
raw_hic_dir = '../../data/hic/'
cell_type = 'IMR90'
resol_str = '250kb'
resol = 250000
quality = 'MAPQGE30'
chr1 = 10
chr2 = 11

In [4]:
# Load raw HiC data
raw_hic_data = phc.get_raw_hic_sparse(raw_hic_dir, cell_type, resol_str, resol, quality, chr1, chr2)
raw_hic_data.head()

locus_chr1  locus_chr2  value  norm_locus_chr1  norm_locus_chr2
0           0           0   15.0         0.537635         0.445645
1           1           0    7.0         1.070878         0.445645
2           2           0    5.0         1.065181         0.445645
3           3           0    2.0         1.035501         0.445645
4           4           0    2.0         1.006773         0.445645

In [5]:
# Normalize HiC data
normalized_hic_data = phc.normalize_raw_hic_sparse(raw_hic_data)
normalized_hic_data.head()

locus_chr1  locus_chr2  value  norm_locus_chr1  norm_locus_chr2  norm_value
0           0           0   15.0         0.537635         0.445645   62.605896
1           1           0    7.0         1.070878         0.445645   14.667944
2           2           0    5.0         1.065181         0.445645   10.533145
3           3           0    2.0         1.035501         0.445645    4.334021
4           4           0    2.0         1.006773         0.445645    4.457689

In [6]:
# Get chromosome sizes
df_sizes = phc.get_chrom_sizes(genome_dir,resol)
chr1_size = int(df_sizes[df_sizes['chr']==str(chr1)]['size_loci'])
chr2_size = int(df_sizes[df_sizes['chr']==str(chr2)]['size_loci'])
df_sizes.head()

chr       size  size_loci  size_roundup
0   1  249250621        998     249500000
1   2  243199373        973     243250000
2   3  198022430        793     198250000
3   4  191154276        765     191250000
4   5  180915260        724     181000000

In [7]:
# Get dense HiC dataframe
df = phc.get_dense_hic_dataframe(normalized_hic_data, chr1_size, chr2_size, resol)
df.head()

0          250000     500000     750000     1000000    1250000    \
0        62.605896  16.397430  16.378051   8.793959   5.582416  18.580250   
250000   14.667944  14.112574  10.571921   8.830019   7.006634   1.332603   
500000   10.533145  16.552739   4.723765  11.096564   8.452935   5.358924   
750000    4.334021   8.513592   4.859160   4.565848   5.796812   5.512525   
1000000   4.457689   5.003727   2.498907   4.696132   4.471665   4.252366   

         1500000    1750000    2000000    2250000      ...      132750000  \
0         6.116312   6.560183  11.188324   0.000000    ...       5.492073   
250000    3.070696   6.587084   3.370261   3.769851    ...       0.919099   
500000    5.145203   5.518598   4.517718   5.685025    ...       0.000000   
750000    4.234142   3.406066   2.323604   5.847972    ...       1.901000   
1000000   3.266220   3.503255   5.974766   2.004947    ...       1.955244   

         133000000  133250000  133500000  133750000  134000000  134250000  \
0         0.000000   0.000000   4.359249   5.574047   3.478282   2.188934   
250000    0.000000   1.071314   2.188562   1.865635   4.365681   3.296866   
500000    0.000000   0.000000   1.100134   4.689035   0.877807   0.000000   
750000    0.000000   0.000000   0.000000   1.929374   4.514835   4.546004   
1000000   3.367685   0.000000   2.327917   4.961068   5.572395   1.168930   

         134500000  134750000  135000000  
0         3.945192   5.378288        0.0  
250000    3.961369   0.000000        0.0  
500000    1.991279   2.714614        0.0  
750000    0.000000   2.792422        0.0  
1000000   5.267008   1.436051        0.0  

[5 rows x 541 columns]

In [8]:
# Plot HiC dense dataframe
plotname = 'norm_hic_'+'chr'+str(chr1)+'_'+'chr'+str(chr2)+'.eps'
hic_plots_dir = '../../save/hic_plots_dir/'
phc.plot_dense_hic_dataframe(df, chr1, chr2, plotname, hic_plots_dir)

In [9]:
# Get centromere locations
df_centrom = phc.get_centromere_locations(genome_dir)
df_centrom.head()

bin chrom  chromStart   chromEnd    ix  n     size        type bridge
0   23  chr1   121535434  124535434  1270  N  3000000  centromere     no
1   20  chr2    92326171   95326171   770  N  3000000  centromere     no
2    2  chr3    90504854   93504854   784  N  3000000  centromere     no
3    1  chr4    49660117   52660117   447  N  3000000  centromere     no
4   14  chr5    46405641   49405641   452  N  3000000  centromere     no

In [10]:
# Filter out centromeres
filter_size = 2000000
df = phc.filter_centromeres(df, chr1, 'row', df_centrom, filter_size, resol)
df = phc.filter_centromeres(df, chr2, 'col', df_centrom, filter_size, resol)

In [11]:
# Plot HiC data after filtering out centromeres and repeats
plotname = 'norm_filtcentromeres_hic_'+'chr'+str(chr1)+'_'+'chr'+str(chr2)+'.eps'
hic_plots_dir = '../../save/hic_plots_dir/'
phc.plot_dense_hic_dataframe(df, chr1, chr2, plotname, hic_plots_dir)

In [12]:
# Load repeats data
df_repeats = phc.load_repeats_data(genome_dir)
df_repeats.head()

genoname  genoStart  genoEnd  repLength
0     chr1      10000    10468        468
1     chr1      10468    11447        979
2     chr1      11503    11675        172
3     chr1      11677    11780        103
4     chr1      15264    15355         91

In [13]:
# Find repeat-covered loci to filter out
chr_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
dic_repeats_tofilter = phc.find_repeat_locations(df_repeats, chr_list, df_sizes, resol)

In [14]:
# Filter repeats for chr1 and chr2
df = phc.filter_repeats(df, chr1, dic_repeats_tofilter, 'row')
df = phc.filter_repeats(df, chr2, dic_repeats_tofilter, 'col')

In [15]:
# Plot HiC data after filtering out centromeres
plotname = 'norm_filtcentromeres_filtrepeats_hic_'+'chr'+str(chr1)+'_'+'chr'+str(chr2)+'.eps'
hic_plots_dir = '../../save/hic_plots_dir/'
phc.plot_dense_hic_dataframe(df, chr1, chr2, plotname, hic_plots_dir)

In [16]:
# Log-transform dataframe
df_transformed = phc.log_transform(df)

In [17]:
# Filter out outliers
df_transformed = phc.filter_outliers(df_transformed)

In [18]:
# Plot HiC data after filtering out centromeres, repeats and outliers
plotname = 'norm_filtcentromeres_filtrepeats_filtoutliers_hic_'+'chr'+str(chr1)+'_'+'chr'+str(chr2)+'.eps'
hic_plots_dir = '../../save/hic_plots_dir/'
phc.plot_dense_hic_dataframe(df_transformed, chr1, chr2, plotname, hic_plots_dir)